In [2]:
import os
import numpy as np
import cv2
import imutils
import random
from math import log10, floor
from matplotlib import pyplot as plt

In [3]:
def crop(img):
    h,w = img.shape[:2]
    imgs=[]
    for i in range(0,w-224,20):
        for j in range(0, h-224,20):
            imgs.append(img[j:j+224, i:i+224])
    return imgs

def fill(img, h, w):
    img = cv2.resize(img, (w, h), cv2.INTER_CUBIC)
    return img
        
def horizontal_shift(img, ratio=0.0):
    if ratio > 1 or ratio < -1:
        print('Value should be less than 1 and greater than 0')
        return img
    h, w = img.shape
    to_shift = w*ratio
    if ratio > 0:
        img = img[:, :int(w-to_shift)]
    if ratio < 0:
        img = img[:, int(-1*to_shift):]  
    img = fill(img, h, w)
    return img

def vertical_shift(img, ratio=0.0):
    if ratio > 1 or ratio < -1:
        print('Value should be less than 1 and greater than 0')
        return img
    h, w = img.shape
    to_shift = h*ratio
    if ratio > 0:
        img = img[:int(h-to_shift), :]
    if ratio < 0:
        img = img[int(-1*to_shift):, :]
    img = fill(img, h, w)
    return img

def zoom(img, value):
    if value > 1 or value < 0:
        print('Value for zoom should be less than 1 and greater than 0')
        return img
    h, w = img.shape
    h_taken = int(value*h)
    w_taken = int(value*w)
    h_start = random.randint(0, h-h_taken)
    w_start = random.randint(0, w-w_taken)
    img = img[h_start:h_start+h_taken, w_start:w_start+w_taken]
    img = fill(img, h, w)
    return img

def horizontal_flip(img):
    return cv2.flip(img, 1)

def vertical_flip(img):
    return cv2.flip(img, 0)

def rotation(img, angle):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))
    return img

def highpass(img, sigma):
    return img - cv2.GaussianBlur(img, (0,0), sigma) + 127

def hist_sliding(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(img)

## Preprocessing
### Crop pics and Augmentation

In [4]:
def preprocessing():
    ### Crop pics
    for i in range(1,7):
        dir_list = os.listdir(f"Dataset2/{i}")
        count=0
        for img_file in dir_list:
            img = cv2.imread(f"Dataset2/{i}/{img_file}")
            imgs = crop(img)

            for k in range(len(imgs)):
                cv2.imwrite(f"Dataset2/{i}/{count}.jpg", imgs[k])
                count+=1

    ### Horizontal Flip
    for i in range(1,7):
        dir_list = os.listdir(f"Dataset2/{i}")
        n=len(dir_list)
        print(i,end=", ")
        for img_file in dir_list:
            img = cv2.imread(f"Dataset2/{i}/{img_file}")
            img = horizontal_flip(img)
            cv2.imwrite(f"Dataset2/{i}/{n+int(img_file.split('.')[0])}.jpg", img)
    print()   
    ### Vertical Flip
    for i in range(1,7):
        dir_list = os.listdir(f"Dataset2/{i}")
        n=len(dir_list)
        print(i, end=", ")
        for img_file in dir_list:
            img = cv2.imread(f"Dataset2/{i}/{img_file}")
            img = horizontal_flip(img)
            cv2.imwrite(f"Dataset2/{i}/{n+int(img_file.split('.')[0])}.jpg", img)
    print()
    ### Rotation by 90 degree
    for i in range(1,7):
        dir_list = os.listdir(f"Dataset2/{i}")
        n=len(dir_list)
        print(i, end=", ")
        for img_file in dir_list:
            img = cv2.imread(f"Dataset2/{i}/{img_file}")
            img = horizontal_flip(img)
            cv2.imwrite(f"Dataset2/{i}/{n+int(img_file.split('.')[0])}.jpg", img)
            
#preprocessing()

## Models
### CNN model for image classification

In [5]:
X=[]; y=[]
for i in range(1,7):
    dir_list = os.listdir(f"Dataset2/{i}")
    for img_file in dir_list:
        X.append(f"Dataset2/{i}/{img_file}")
        y.append(i)
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(362880,) (362880,)


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2)
print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(232243,) (58061,) (72576,) (232243,) (58061,) (72576,)


In [9]:
for i in range(30000):
    img = cv2.imread(X_train[i])
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = hist_sliding(img)
    img = highpass(img, 5)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite(f"Dataset2/train/{y_train[i]}/{i}.jpg", img)

for i in range(3000):
    img = cv2.imread(X_val[i])
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = hist_sliding(img)
    img = highpass(img, 5)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite(f"Dataset2/val/{y_val[i]}/{i}.jpg", img)

for i in range(3000):
    img = cv2.imread(X_test[i])
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = hist_sliding(img)
    img = highpass(img, 5)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    cv2.imwrite(f"Dataset2/test/{y_test[i]}/{i}.jpg", img)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

# create a new generator
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory("Dataset2/train", class_mode="categorical", shuffle=False, batch_size=32, target_size=(224, 224))
val = imagegen.flow_from_directory("Dataset2/val/", class_mode="categorical", shuffle=False, batch_size=32, target_size=(224, 224))
test = imagegen.flow_from_directory("Dataset2/test/", class_mode="categorical", shuffle=False, batch_size=32, target_size=(224, 224))

2022-05-17 02:41:12.068052: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/naveen/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-17 02:41:12.068095: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Found 30000 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [13]:
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

# build a sequential model
model = Sequential()
model.add(InputLayer(input_shape=(224, 224, 3)))

# 1st conv block
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=6, activation='softmax'))

# compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fit on data for 5 epochs
checkpoint = ModelCheckpoint('Models2/cnn{epoch:01d}.h5', save_freq=938)
model.fit(train, epochs=5, validation_data=val, callbacks=[checkpoint])

2022-05-17 02:43:12.637532: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-17 02:43:12.637589: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (naveen): /proc/driver/nvidia/version does not exist
2022-05-17 02:43:12.662234: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5


2022-05-17 02:43:19.919573: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 19267584 exceeds 10% of free system memory.
2022-05-17 02:43:20.005906: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 160563200 exceeds 10% of free system memory.
2022-05-17 02:43:20.915421: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 40140800 exceeds 10% of free system memory.
2022-05-17 02:43:20.946427: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 20070400 exceeds 10% of free system memory.
2022-05-17 02:43:21.884031: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 20815200 exceeds 10% of free system memory.


938/938 [==============================] - 1107s 1s/step - loss: 0.5258 - accuracy: 0.8065 - val_loss: 4.8865 - val_accuracy: 0.3717
Epoch 2/5
938/938 [==============================] - 1297s 1s/step - loss: 0.1208 - accuracy: 0.9704 - val_loss: 2.6420 - val_accuracy: 0.5263
Epoch 3/5
938/938 [==============================] - 1090s 1s/step - loss: 0.0137 - accuracy: 0.9961 - val_loss: 4.0997 - val_accuracy: 0.5133
Epoch 4/5
938/938 [==============================] - 895s 955ms/step - loss: 0.0719 - accuracy: 0.9815 - val_loss: 6.7602 - val_accuracy: 0.5113
Epoch 5/5
938/938 [==============================] - 830s 885ms/step - loss: 0.0122 - accuracy: 0.9966 - val_loss: 11.5042 - val_accuracy: 0.4537


In [21]:
from tensorflow.keras.utils import to_categorical
from keras.models import load_model

y_test = to_categorical(test.labels)
y_test = np.argmax(y_test, axis=1)

filenames = test.filenames
nb_samples = len(filenames)
model = load_model("Models2/cnn5.h5")
y_pred = model.predict(test, steps=np.ceil(nb_samples/32))
y_pred = np.argmax(y_pred,axis=1)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print(classification_report(y_test, y_pred, target_names=["1","2","3","4","5","6"], digits=5))
print(confusion_matrix(y_test, y_pred))
print("ACCURACY OF THE MODEL: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1    0.59524   0.32258   0.41841       310
           2    0.00000   0.00000   0.00000       320
           3    0.00000   0.00000   0.00000       552
           4    0.46154   0.66667   0.54545       612
           5    0.34089   0.85251   0.48704       617
           6    0.82957   0.56197   0.67004       589

    accuracy                        0.45500      3000
   macro avg    0.37121   0.40062   0.35349      3000
weighted avg    0.38865   0.45500   0.38623      3000

[[100   0   0  26 180   4]
 [  0   0   0 320   0   0]
 [  0   0   0   0 552   0]
 [  0   0   0 408 204   0]
 [ 11   0   4  12 526  64]
 [ 57   0   2 118  81 331]]
ACCURACY OF THE MODEL:  0.455


/home/naveen/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/naveen/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/naveen/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
